In [49]:
import sys
import time
import requests
import json
from datetime import datetime, timedelta
import pandas as pd
import geopandas as gpd 

sys.path.append("../")
from config.config import GEOSPHERE_INCA_TS_URL
from config.config import BASE_PATH, PATH_TO_PATH_CONFIG_FILE
from src.utils import load_paths_from_yaml, replace_base_path, calculate_date_of_interest_x_hours_before

In [50]:
# Load paths from the YAML file
paths = load_paths_from_yaml(PATH_TO_PATH_CONFIG_FILE)
paths = replace_base_path(paths, BASE_PATH)

In [77]:

def get_geosphere_data(parameters: list = ["T2M", "RR"],
                       start: str = '2021-08-01T00:00', 
                       end: str = '2021-08-01T00:00', 
                       lat_lon: list = ['48.206248, 16.367569'], 
                       output_format: str = "geojson"):
    """
    function for retrieving data from geosphere API
    """
    
    lat_lon_params = '&lat_lon='.join(lat_lon)

    parameter_string_for_url = ""
    for a in parameters:
        parameter_string_for_url += f"parameters={a}&"
    
    url = f'{GEOSPHERE_INCA_TS_URL}?{parameter_string_for_url[:-1]}&start={start}&end={end}&lat_lon={lat_lon_params}&output_format={output_format}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f'Request failed with status code {response.status_code}.')

def get_lat_lon(row: pd.Series):
    point = (row["geometry"].y, row["geometry"].x)
    return point

def get_time_range(row: pd.Series, days: int):
    end = row.date
    start = end - timedelta(days=days)
    return start.isoformat(), end.isoformat()

def transform_date(input_date):
    """transforms date from "%m/%d/%Y" into '%Y-%m-%dT12:00' format"""
    date_object = datetime.strptime(input_date, "%m/%d/%Y")
    output_date = date_object.strftime('%Y-%m-%dT12:00')
    return str(output_date)

In [78]:
# loading fire dataset
event_data = gpd.read_file(paths["fire_events"]["final"])
event_data["date"] = event_data["date"].apply(transform_date)
event_data.to_crs("EPSG:4326", inplace=True)

In [79]:
with open(paths["inca"]["training_data"]["T2M"], 'a') as f:

    for i, row in event_data.iterrows():
        transformed_point = get_lat_lon(row)
        lat_lon = [f"{transformed_point[0]}, {transformed_point[1]}"]
        start_date = calculate_date_of_interest_x_hours_before(row.date, hours=24)
        data = get_geosphere_data(["T2M", "RR", "UU", "VV", "RH2M"], start_date, row.date, lat_lon, "geojson")
        json.dump(data, f)
        f.write('\n')
        print(i)

        time.sleep(5)

f.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
